In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.1 MB/s 
     |████████████████████████████████| 86 kB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 18.4 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import re
import time
import string
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm, notebook
import pickle
from joblib import dump
from sklearn.model_selection import train_test_split

from keras.layers import Dense, GlobalMaxPooling1D, Dropout, Dense, BatchNormalization, Input
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM,Bidirectional
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn import datasets, svm
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/PhoBert/"

df = pd.read_xlsx(data_path,name = "")

#convert df_content to data

**PhoBert Embedding**

In [7]:
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer
phobert, tokenizer = load_bert()

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def phobert_embed_sentence(padded, mask, model=phobert):
    # embed a single setence data
    # param padded: a tokenized, padded sentence
    # param mask: an attention mask of the padded sentence
    padded = torch.tensor(padded).to(torch.long)
    mask = torch.tensor(mask)
    with torch.no_grad():
        last_hidden_states = model(input_ids=padded, attention_mask=mask)[0]
    vector = last_hidden_states[:, 0, :].numpy() # [:,0,:] to get embedding vector of the first output token [CLS]
    return vector.flatten()

def phobert_embed_data(data, tokenizer=tokenizer):
    # embed the whole dataset with phobert
    # param padded_data: tokenized, padded dataset
    # param mask_data: attention masks of the padded dataset
    MAX_LENGTH = tokenizer.model_max_length # phobert default max sequence length = 256

    embedded_data = np.array([])
    for line in tqdm(data):
        tokenized_line = tokenizer.encode(line, max_length=MAX_LENGTH, truncation=True)
        # pad sentence to a pre-defined max length, no truncating since it is already truncated in the phobert tokenizing
        padded_line = pad_sequences([tokenized_line], maxlen=MAX_LENGTH, padding='post', value=1)
        # Get attention mask from padded sentence of data to make PhoBERT focus on non-padded data only
        # pad tokenized sentence with value = 1, since 1 is pre-defined padding value of PhoBERT
        mask = np.where(padded_line == 1, 0, 1)

        embedded_line = phobert_embed_sentence(padded_line, mask)
        
        if embedded_data.shape[0] == 0:
            embedded_data = np.empty((0, embedded_line.shape[0]), 'float32')
            
        embedded_data = np.concatenate((embedded_data, [embedded_line]))
    return embedded_data

In [ ]:
embedding = phobert_embed_data("data")

**Cleaning Data**

In [43]:
# !pip install underthesea
# !git clone https://github.com/stopwords/vietnamese-stopwords
import underthesea
stopwords_path = "/content/vietnamese-stopwords/vietnamese-stopwords.txt"

def standardize_data(sentence):
    row = re.sub(r"[\.,\?]+$-","",sentence)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

def cleaning_text(text, keep_punct=True):
    if keep_punct:
        return re.sub(u'[^{Latin}0-9[:punct:]]+', u' ', text)
    return re.sub(u'[^{Latin}0-9]+', u' ', text)

def load_stopwords():
    sw = []
    with open(stopwords_path, encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

sw = load_stopwords()
def preprocessing(text, keep_punct=True):
    text = standardize_data(text)
    text = remove_emoji(text)
    #tokenize
    line = underthesea.word_tokenize(text)

    #filter stopword
    filter_words = [word for word in line if word not in sw]
    #concat
    line = "".join(text)
    line = underthesea.word_tokenize(line,format = "text")
    
    return cleaning_text(line)

**Load model**

In [9]:
MLP_model = tf.keras.models.load_model("/content/drive/MyDrive/PhoBert/MLP")

In [57]:
def predict(sentence):

    x = [cleaning_text(sentence)]
    
    doc = phobert_embed_data(x)
    res = MLP_model.predict(np.array(np.expand_dims(doc,-1)))[0]
    print('\n')
    print(res)
    if res[0] > res[1]:
       return "negative"
    return "positive"

In [58]:
sentence = "áo này đẹp ghê"
predict(sentence)

100%|██████████| 1/1 [00:00<00:00,  1.04it/s]



[0.23089075 0.7691092 ]


'positive'

In [59]:
sentence = "cổ tay ko vừa"
predict(sentence)

100%|██████████| 1/1 [00:01<00:00,  1.65s/it]



[0.906933   0.09306694]


'negative'

In [60]:
sentence = "dịch vụ khá tốt, tôi rất hài lòng"
predict(sentence)

100%|██████████| 1/1 [00:01<00:00,  1.55s/it]



[0.14915468 0.8508453 ]


'positive'

In [62]:
sentence = "mặt hàng còn xấu, đề nghị chỉnh sửa"
predict(sentence)

100%|██████████| 1/1 [00:01<00:00,  1.39s/it]



[0.6910521  0.30894786]


'negative'

**With data**

In [41]:
data_path = "/content/drive/MyDrive/PhoBert/CRAWL_20220614-161812_YOUTUBE.csv"

df = pd.read_csv(data_path)

In [53]:
df["PREDICT"] = df["comment_post"].map(lambda x : predict(x))

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


[0.14215375 0.8578462 ]


100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


[0.05073089 0.9492692 ]


100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


...

[0.5781838 0.4218162]


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


[0.7645963  0.23540372]


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

[0.06568909 0.9343109 ]


In [56]:
df.PREDICT.value_counts() ##=>áo này không có được đón nhận cho lắm

negative    51
positive    45
Name: PREDICT, dtype: int64